# Cryptocurrency Data Producer

## Real-Time Data Ingestion from Binance WebSocket API

This notebook connects to Binance's public WebSocket API and streams real cryptocurrency
trades into a Unity Catalog Volume. The data is written as JSON files that can be
consumed by Auto Loader for streaming analytics.

### How It Works:
1. **Connect** to Binance WebSocket for multiple trading pairs (BTC, ETH, etc.)
2. **Collect** trades in memory for a configurable batch window
3. **Write** batched trades as JSON files to Unity Catalog Volume
4. **Repeat** until stopped or time limit reached

### Usage:
- **Demo Mode**: Run interactively for 2-5 minutes to collect sample data
- **Job Mode**: Schedule as a Databricks Job for continuous ingestion

### Data Source:
- **API**: Binance.US WebSocket (wss://stream.binance.us)
- **Streams**: Real-time trade data for major cryptocurrency pairs
- **Volume**: ~50-200 trades per second across all pairs
- **Cost**: FREE (no API key required for public streams)

## Setup & Configuration

In [0]:
# Install required packages
%pip install websocket-client

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
# Configuration
CATALOG = "takamol_demo"
SCHEMA = "crypto_streaming"
VOLUME_NAME = "crypto_landing"

# Trading pairs to monitor (Binance format)
TRADING_PAIRS = [
    "btcusdt",   # Bitcoin
    "ethusdt",   # Ethereum
    "bnbusdt",   # Binance Coin
    "solusdt",   # Solana
    "xrpusdt",   # Ripple
]

# Producer settings
# For live demos: use BATCH_INTERVAL_SECONDS = 3 and RUN_DURATION_MINUTES = 10-15
BATCH_INTERVAL_SECONDS = 3     # How often to write files (3s for demos, 5s for production)
RUN_DURATION_MINUTES = 3      # How long to run (set to None for continuous)
MAX_TRADES_PER_BATCH = 1000    # Safety limit per batch file

print("=" * 60)
print("CRYPTOCURRENCY DATA PRODUCER")
print("=" * 60)
print(f"\nConfiguration:")
print(f"  Catalog: {CATALOG}")
print(f"  Schema: {SCHEMA}")
print(f"  Volume: {VOLUME_NAME}")
print(f"  Trading Pairs: {', '.join([p.upper() for p in TRADING_PAIRS])}")
print(f"  Batch Interval: {BATCH_INTERVAL_SECONDS} seconds")
print(f"  Run Duration: {RUN_DURATION_MINUTES} minutes" if RUN_DURATION_MINUTES else "  Run Duration: Continuous")

CRYPTOCURRENCY DATA PRODUCER

Configuration:
  Catalog: takamol_demo
  Schema: crypto_streaming
  Volume: crypto_landing
  Trading Pairs: BTCUSDT, ETHUSDT, BNBUSDT, SOLUSDT, XRPUSDT
  Batch Interval: 3 seconds
  Run Duration: 3 minutes


## Create Unity Catalog Resources

In [0]:
# Create schema if not exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")

# Create volume for landing zone
spark.sql(f"""
    CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME_NAME}
    COMMENT 'Landing zone for real-time cryptocurrency trade data from Binance'
""")

VOLUME_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME_NAME}"
LANDING_PATH = f"{VOLUME_PATH}/trades"

# Create subdirectory for trades
try:
    dbutils.fs.mkdirs(LANDING_PATH)
except:
    pass  # Already exists

print(f"\n✓ Unity Catalog resources ready")
print(f"  Landing Path: {LANDING_PATH}")

# Check existing files
try:
    existing_files = dbutils.fs.ls(LANDING_PATH)
    print(f"  Existing Files: {len(existing_files)}")
except:
    print(f"  Existing Files: 0 (new landing zone)")


✓ Unity Catalog resources ready
  Landing Path: /Volumes/takamol_demo/crypto_streaming/crypto_landing/trades
  Existing Files: 80


## WebSocket Producer Class

In [0]:
import websocket
import json
import threading
import time
from datetime import datetime
from collections import deque
import uuid

class BinanceTradeProducer:
    """
    Connects to Binance WebSocket and writes trade data to Unity Catalog Volume.

    Features:
    - Multi-stream connection (multiple trading pairs)
    - Automatic batching and file writing
    - Reconnection on disconnect
    - Thread-safe trade collection
    """

    def __init__(self, trading_pairs, landing_path, batch_interval=5, max_trades_per_batch=1000):
        self.trading_pairs = trading_pairs
        self.landing_path = landing_path
        self.batch_interval = batch_interval
        self.max_trades_per_batch = max_trades_per_batch

        # Thread-safe trade buffer
        self.trade_buffer = deque(maxlen=10000)
        self.buffer_lock = threading.Lock()

        # Statistics
        self.stats = {
            "trades_received": 0,
            "batches_written": 0,
            "files_created": 0,
            "connection_errors": 0,
            "start_time": None,
            "last_trade_time": None
        }

        # Control flags
        self.running = False
        self.ws = None
        self.writer_thread = None

    def _build_stream_url(self):
        """Build combined stream URL for multiple pairs."""
        streams = "/".join([f"{pair}@trade" for pair in self.trading_pairs])
        return f"wss://stream.binance.us:9443/stream?streams={streams}"

    def _on_message(self, ws, message):
        """Handle incoming WebSocket message."""
        try:
            data = json.loads(message)

            # Combined stream format wraps data in {"stream": ..., "data": ...}
            if "data" in data:
                trade = data["data"]
            else:
                trade = data

            # Only process trade events
            if trade.get("e") == "trade":
                # Enrich with processing metadata
                enriched_trade = {
                    "event_type": trade["e"],
                    "event_time": trade["E"],
                    "symbol": trade["s"],
                    "trade_id": trade["t"],
                    "price": float(trade["p"]),
                    "quantity": float(trade["q"]),
                    "buyer_order_id": trade["b"],
                    "seller_order_id": trade["a"],
                    "trade_time": trade["T"],
                    "is_buyer_maker": trade["m"],
                    "trade_value_usdt": float(trade["p"]) * float(trade["q"]),
                    "ingestion_time": int(time.time() * 1000),
                    "producer_id": str(uuid.uuid4())[:8]
                }

                with self.buffer_lock:
                    self.trade_buffer.append(enriched_trade)
                    self.stats["trades_received"] += 1
                    self.stats["last_trade_time"] = datetime.now()

        except Exception as e:
            print(f"  Error processing message: {e}")

    def _on_error(self, ws, error):
        """Handle WebSocket error."""
        print(f"  WebSocket Error: {error}")
        self.stats["connection_errors"] += 1

    def _on_close(self, ws, close_status_code=None, close_msg=None):
        """Handle WebSocket close."""
        print(f"  WebSocket Closed: {close_status_code} - {close_msg}")

    def _on_open(self, ws):
        """Handle WebSocket connection open."""
        print(f"  WebSocket Connected to Binance!")
        print(f"  Streaming: {', '.join([p.upper() for p in self.trading_pairs])}")

    def _batch_writer(self):
        """Background thread that periodically writes batches to files."""
        while self.running:
            time.sleep(self.batch_interval)

            if not self.running:
                break

            # Collect trades from buffer
            trades_to_write = []
            with self.buffer_lock:
                while self.trade_buffer and len(trades_to_write) < self.max_trades_per_batch:
                    trades_to_write.append(self.trade_buffer.popleft())

            if trades_to_write:
                self._write_batch(trades_to_write)

    def _write_batch(self, trades):
        """Write a batch of trades to a JSON file."""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            batch_id = str(uuid.uuid4())[:8]
            filename = f"trades_{timestamp}_{batch_id}.json"
            filepath = f"{self.landing_path}/{filename}"

            # Create JSON content (newline-delimited JSON for better streaming)
            json_content = "\n".join([json.dumps(trade) for trade in trades])

            # Write using dbutils
            dbutils.fs.put(filepath, json_content, overwrite=True)

            self.stats["batches_written"] += 1
            self.stats["files_created"] += 1

            # Print progress
            symbols = set(t["symbol"] for t in trades)
            avg_price_btc = sum(t["price"] for t in trades if t["symbol"] == "BTCUSDT") / max(1, sum(1 for t in trades if t["symbol"] == "BTCUSDT"))

            print(f"  Batch {self.stats['batches_written']:3d}: {len(trades):4d} trades | "
                  f"Symbols: {', '.join(sorted(symbols))} | "
                  f"File: {filename}")

        except Exception as e:
            print(f"  Error writing batch: {e}")

    def start(self, duration_minutes=None):
        """Start the producer."""
        print("\n" + "=" * 60)
        print("STARTING BINANCE TRADE PRODUCER")
        print("=" * 60)

        self.running = True
        self.stats["start_time"] = datetime.now()

        # Start batch writer thread
        self.writer_thread = threading.Thread(target=self._batch_writer, daemon=True)
        self.writer_thread.start()

        # Build WebSocket URL
        ws_url = self._build_stream_url()
        print(f"\nConnecting to: {ws_url[:50]}...")

        # Create WebSocket connection
        self.ws = websocket.WebSocketApp(
            ws_url,
            on_message=self._on_message,
            on_error=self._on_error,
            on_close=self._on_close,
            on_open=self._on_open
        )

        # Run WebSocket in background thread
        ws_thread = threading.Thread(target=self.ws.run_forever, daemon=True)
        ws_thread.start()

        # Calculate end time
        end_time = None
        if duration_minutes:
            end_time = time.time() + (duration_minutes * 60)
            print(f"\nRunning for {duration_minutes} minutes...")
        else:
            print("\nRunning continuously (interrupt to stop)...")

        print("-" * 60)

        try:
            while self.running:
                time.sleep(1)

                # Check if duration exceeded
                if end_time and time.time() >= end_time:
                    print("\n[Duration limit reached]")
                    break

        except KeyboardInterrupt:
            print("\n[Interrupted by user]")

        self.stop()

    def stop(self):
        """Stop the producer gracefully."""
        print("\n" + "-" * 60)
        print("STOPPING PRODUCER...")

        self.running = False

        # Close WebSocket
        if self.ws:
            self.ws.close()

        # Write remaining trades
        remaining_trades = []
        with self.buffer_lock:
            while self.trade_buffer:
                remaining_trades.append(self.trade_buffer.popleft())

        if remaining_trades:
            print(f"  Writing {len(remaining_trades)} remaining trades...")
            self._write_batch(remaining_trades)

        # Print final statistics
        duration = (datetime.now() - self.stats["start_time"]).total_seconds() if self.stats["start_time"] else 0

        print("\n" + "=" * 60)
        print("PRODUCER STATISTICS")
        print("=" * 60)
        print(f"  Duration: {duration:.1f} seconds")
        print(f"  Trades Received: {self.stats['trades_received']:,}")
        print(f"  Batches Written: {self.stats['batches_written']}")
        print(f"  Files Created: {self.stats['files_created']}")
        print(f"  Trades/Second: {self.stats['trades_received'] / max(1, duration):.1f}")
        print(f"  Connection Errors: {self.stats['connection_errors']}")
        print("=" * 60)

        return self.stats

## Run the Producer

The producer will connect to Binance and start collecting real trades.

- In **demo mode**, it runs for a few minutes then stops
- As a **Databricks Job**, set `RUN_DURATION_MINUTES = None` for continuous operation

In [0]:
# Create and run the producer
producer = BinanceTradeProducer(
    trading_pairs=TRADING_PAIRS,
    landing_path=LANDING_PATH,
    batch_interval=BATCH_INTERVAL_SECONDS,
    max_trades_per_batch=MAX_TRADES_PER_BATCH
)

# Start collecting data
stats = producer.start(duration_minutes=RUN_DURATION_MINUTES)


STARTING BINANCE TRADE PRODUCER

Connecting to: wss://stream.binance.us:9443/stream?streams=btcusd...

Running for 3 minutes...
------------------------------------------------------------
  WebSocket Connected to Binance!
  Streaming: BTCUSDT, ETHUSDT, BNBUSDT, SOLUSDT, XRPUSDT
Wrote 339 bytes.
  Batch   1:    1 trades | Symbols: ETHUSDT | File: trades_20260112_062345_0e9781bf.json
Wrote 339 bytes.
  Batch   2:    1 trades | Symbols: BTCUSDT | File: trades_20260112_062352_11a1137f.json
Wrote 343 bytes.
  Batch   3:    1 trades | Symbols: ETHUSDT | File: trades_20260112_062422_0e2df7f6.json

[Duration limit reached]

------------------------------------------------------------
STOPPING PRODUCER...
  WebSocket Closed: None - None

PRODUCER STATISTICS
  Duration: 180.3 seconds
  Trades Received: 3
  Batches Written: 3
  Files Created: 3
  Trades/Second: 0.0
  Connection Errors: 0


## Verify Data Collection

In [0]:
# List files in landing zone
print(f"Files in Landing Zone ({LANDING_PATH}):")
print("-" * 60)

try:
    files = dbutils.fs.ls(LANDING_PATH)
    files_sorted = sorted(files, key=lambda x: x.name, reverse=True)

    total_size = 0
    for f in files_sorted[:20]:  # Show last 20 files
        size_kb = f.size / 1024
        total_size += f.size
        print(f"  {f.name:45} {size_kb:8.1f} KB")

    if len(files) > 20:
        print(f"  ... and {len(files) - 20} more files")

    print("-" * 60)
    print(f"Total Files: {len(files)}")
    print(f"Total Size: {total_size / 1024 / 1024:.2f} MB")

except Exception as e:
    print(f"No files found: {e}")

Files in Landing Zone (/Volumes/takamol_demo/crypto_streaming/crypto_landing/trades):
------------------------------------------------------------
  trades_20260112_062422_0e2df7f6.json               0.3 KB
  trades_20260112_062352_11a1137f.json               0.3 KB
  trades_20260112_062345_0e9781bf.json               0.3 KB
  trades_20260112_060648_56c5362e.json               0.3 KB
  trades_20260112_060621_463b1576.json               0.3 KB
  trades_20260112_060611_c9c82726.json               0.3 KB
  trades_20260112_055428_8763e3e0.json               0.3 KB
  trades_20260112_055352_f1f8d00e.json               0.7 KB
  trades_20260112_054759_f3781d2b.json               0.3 KB
  trades_20260112_054720_954624f1.json               0.3 KB
  trades_20260112_054717_42de69cf.json               0.3 KB
  trades_20260112_054713_c895b59a.json               0.3 KB
  trades_20260112_054649_62a84f2d.json               0.3 KB
  trades_20260112_054543_b26e5e0f.json               0.3 KB
  trades_2026

## Preview Sample Data

In [0]:
# Read a sample of the collected data
sample_df = (
    spark.read
    .json(LANDING_PATH)
    .limit(100)
)

print(f"Sample Trade Data:")
print(f"Total Records in Sample: {sample_df.count()}")
sample_df.printSchema()

Sample Trade Data:
Total Records in Sample: 94
root
 |-- buyer_order_id: long (nullable = true)
 |-- event_time: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- ingestion_time: long (nullable = true)
 |-- is_buyer_maker: boolean (nullable = true)
 |-- price: double (nullable = true)
 |-- producer_id: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- seller_order_id: long (nullable = true)
 |-- symbol: string (nullable = true)
 |-- trade_id: long (nullable = true)
 |-- trade_time: long (nullable = true)
 |-- trade_value_usdt: double (nullable = true)



In [0]:
# Display sample trades
display(sample_df.orderBy("trade_time", ascending=False))

buyer_order_id,event_time,event_type,ingestion_time,is_buyer_maker,price,producer_id,quantity,seller_order_id,symbol,trade_id,trade_time,trade_value_usdt
773508727,1768198008579,trade,1768198008612,false,142.98,aad48f40,0.04,773508614,SOLUSDT,11527268,1768198008579,5.7192
773508658,1768197980419,trade,1768197980451,false,142.98,9e94bea2,0.07,773508614,SOLUSDT,11527267,1768197980419,10.0086
773508625,1768197971270,trade,1768197971301,false,142.95,b4efbdd8,0.229,773508620,SOLUSDT,11527266,1768197971270,32.735549999999996
1640956542,1768197266878,trade,1768197266918,true,91933.94,4130170e,2.2E-4,1640956556,BTCUSDT,31006074,1768197266878,20.2254668
1347790196,1768197230677,trade,1768197230712,true,3156.87,58148f47,0.0999,1347790198,ETHUSDT,18131381,1768197230676,315.371313
1347790196,1768197230677,trade,1768197230712,true,3156.87,23c22144,1.5093,1347783997,ETHUSDT,18131382,1768197230676,4764.663891
316999567,1768196878690,trade,1768196878724,true,2.0828,c52289cf,225.0,316999572,XRPUSDT,2238301,1768196878690,468.63000000000005
1640951558,1768196837176,trade,1768196837210,true,92248.0,a204444a,2.9E-4,1640951561,BTCUSDT,31006067,1768196837175,26.751920000000002
316999520,1768196833993,trade,1768196834026,true,2.0824,5a38d516,14.4,316999521,XRPUSDT,2238300,1768196833993,29.986559999999997
316999518,1768196832674,trade,1768196832707,true,2.0823,4e23f5ac,141.0,316999519,XRPUSDT,2238299,1768196832673,293.6043


In [0]:
# # Quick statistics
# from pyspark.sql.functions import *

# print("Quick Statistics from Collected Data:")
# print("-" * 60)

# stats_df = (
#     spark.read.json(LANDING_PATH)
#     .groupBy("symbol")
#     .agg(
#         count("*").alias("trade_count"),
#         round(avg("price"), 2).alias("avg_price"),
#         round(min("price"), 2).alias("min_price"),
#         round(max("price"), 2).alias("max_price"),
#         round(sum("trade_value_usdt"), 2).alias("total_volume_usdt")
#     )
#     .orderBy("trade_count", ascending=False)
# )

# display(stats_df)

## Next Steps

The data is now landing in Unity Catalog Volume. To process it as a stream:

1. Open the **`crypto_streaming_analytics`** notebook
2. Configure it to read from the same landing path
3. Run the streaming analytics while this producer continues (or after it stops)

### Production Deployment

To run this continuously as a Databricks Job:

1. Set `RUN_DURATION_MINUTES = None` in configuration
2. Create a new Databricks Job with this notebook
3. Configure appropriate cluster (small single-node is sufficient)
4. Set up alerting for job failures

---

**Takamol Demo - Real-Time Cryptocurrency Data Producer**

*Powered by Binance WebSocket API*